# Text Analysis for Women's E-Commerce Clothing Reviews

## Libraries and Settings

In [1]:
import os
import pandas as pd
from pathlib import Path

from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

%config Completer.use_jedi = False

## Variables

In [2]:
RAW_DIR = os.path.join(os.pardir, 'data', 'raw')
RAW_DATA = os.path.join(RAW_DIR, 'Womens Clothing E-Commerce Reviews.csv')
INTERIM_DIR = os.path.join(os.pardir, 'data', 'interim')
RANDOM_STATE = 8

## Data Ingestion

### Extract

In [3]:
import kaggle
kaggle.api.authenticate()
kaggle.api.dataset_download_files('nicapotato/womens-ecommerce-clothing-reviews', 
                                  path=RAW_DIR,
                                  unzip=True)

In [4]:
raw_data = pd.read_csv(RAW_DATA)

In [5]:
raw_data.head(5)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [6]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


In [7]:
raw_data.describe()

,Unnamed: 0,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count
count,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000
mean,11742.500000,918.118709,43.198544,4.196032,0.822362,2.535936
std,6779.968547,203.298980,12.279544,1.110031,0.382216,5.702202
min,0.000000,0.000000,18.000000,1.000000,0.000000,0.000000
25%,5871.250000,861.000000,34.000000,4.000000,1.000000,0.000000
50%,11742.500000,936.000000,41.000000,5.000000,1.000000,1.000000
75%,17613.750000,1078.000000,52.000000,5.000000,1.000000,3.000000
max,23485.000000,1205.000000,99.000000,5.000000,1.000000,122.000000


### Transform

In [8]:
inter_data = raw_data.copy()

# Reformat column names
columns_formatted = [col.lower().replace(" ", "_") for col in inter_data.columns]
inter_data.columns = columns_formatted

# Select column for classification
inter_data = inter_data[['clothing_id', 'review_text', 'recommended_ind']]

# Handling with missing data: Because 3.6% of missing values, just drop them
inter_data = inter_data[~inter_data['review_text'].isnull()]

# Unbalanced dataset
y = inter_data['recommended_ind']
X = inter_data[inter_data.columns.difference(['recommended_ind'])]
ros = RandomOverSampler(random_state=RANDOM_STATE)
X_over, y_over = ros.fit_resample(X, y)

In [9]:
# Train, Test and Validation splitting

X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

### "Load"

In [10]:
X_dfs = [X_train, X_test, X_val]
y_dfs = [y_train, y_test, y_val]

p = Path(INTERIM_DIR)
if not p.exists():
    os.mkdir(p)
df_names = ['train_interim.csv', 'test_interim.csv', 'val_interim.csv']
for x_df, y_df, df_name in zip(X_dfs, y_dfs, df_names):
    df = pd.merge(x_df, y_df, how="left", left_index=True, right_index=True)
    df.to_csv(os.path.join(INTERIM_DIR, df_name), index=False)

## Comments